In [10]:
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
from collections import Counter
from multimodal.multimodal_data_module import MultiModalSAYCamDataModule
from multimodal.multimodal import TextEncoder, MultiModalModel
from multimodal.multimodal_lit import MultiModalLitModel
from multimodal.lm_lit import LMLitModel
from train import _setup_parser

In [11]:
parser = _setup_parser()
args = parser.parse_args("")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = MultiModalSAYCamDataModule(args)
data.prepare_data()
data.setup()
checkpoint = "checkpoints/lm_embedding_type_spatial_text_encoder_lstm_dropout_o_0.3_batch_size_16/epoch=99.ckpt"
lit_model = LMLitModel.load_from_checkpoint(checkpoint, map_location=device)
#spatial_embedding_checkpoint = "/misc/vlgscratch4/LakeGroup/shared_data/S_multimodal/checkpoints/multimodal-epoch=99.ckpt"
#lit_model = MultiModalLitModel.load_from_checkpoint(spatial_embedding_checkpoint, map_location=device)
vocab = lit_model.vocab
vocab_size = len(vocab)
print(f'vocab_size = {vocab_size}')
for idx in range(vocab_size):
    assert idx in lit_model.idx2word
lit_model.eval()
text_encoder = lit_model.text_encoder
#text_encoder = lit_model.model.text_embed

Calling prepare_data!
SAYCam transcripts have already been downloaded. Skipping this step.
Transcripts have already been renamed. Skipping this step.
Transcripts have already been preprocessed. Skipping this step.
Training frames have already been extracted. Skipping this step.
Training metadata files have already been created . Skipping this step.
Evaluation frames have already been extracted. Skipping this step.
Evaluation metadata files have already been created . Skipping this step.
Vocabulary file already exists. Skipping this step.
Calling setup!
vocab_size = 5589


In [9]:
from tqdm import tqdm

sum_vectors = torch.zeros(vocab_size, text_encoder.hidden_dim, device=device)
cnt_vectors = torch.zeros(vocab_size, dtype=torch.int, device=device)

for x, y, y_len in tqdm(data.train_dataloader()):
    y, y_len = y.cuda(), y_len.cuda()
    outputs = text_encoder(y, y_len)
    for i in range(len(y)):
        for j in range(y_len[i].item()):
            token_id = y[i, j]
            vector = outputs[i, j].detach()
            cnt_vectors[token_id] += 1
            sum_vectors[token_id] += vector

mean_vectors = sum_vectors / cnt_vectors.unsqueeze(-1)
vectors = mean_vectors.cpu().numpy()
for i in range(len(cnt_vectors)):
    if cnt_vectors[i].item() == 0:
        vectors[i].fill(0.)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 8289/8289 [01:26<00:00, 95.94it/s]


In [15]:
tokens = input().split()
token_ids = []
print('not found:', end='')
for s in tokens:
    try:
        token_ids.append(lit_model.word2idx[s])
    except KeyError:
        print(f' {s}')
print()
np.set_printoptions(precision=2)
V = vectors[token_ids]
V = V / np.linalg.norm(V, axis=1, keepdims=True)
print(V @ V.T)
# do did go went play played get got eat ate have had look looked fly flew drive drove stand stood crawl crawled
# do doing go going play playing get getting eat eating have having look looking fly flying drive driving stand standing crawl crawling

red orange yellow green blue purple brown black white
not found:
[[1.   0.78 0.76 0.78 0.86 0.65 0.54 0.63 0.5 ]
 [0.78 1.   0.68 0.74 0.77 0.68 0.61 0.69 0.58]
 [0.76 0.68 1.   0.78 0.77 0.69 0.48 0.55 0.48]
 [0.78 0.74 0.78 1.   0.82 0.75 0.55 0.62 0.59]
 [0.86 0.77 0.77 0.82 1.   0.69 0.62 0.65 0.52]
 [0.65 0.68 0.69 0.75 0.69 1.   0.44 0.59 0.42]
 [0.54 0.61 0.48 0.55 0.62 0.44 1.   0.72 0.43]
 [0.63 0.69 0.55 0.62 0.65 0.59 0.72 1.   0.53]
 [0.5  0.58 0.48 0.59 0.52 0.42 0.43 0.53 1.  ]]


In [5]:
tokens = input().split()
assert len(tokens) % 2 == 0
token_ids = []
ok = True
print('not found:', end='')
for s in tokens:
    try:
        token_ids.append(lit_model.word2idx[s])
    except KeyError:
        print(f' {s}')
        ok = False
print()
if ok:
    np.set_printoptions(precision=2)
    V = vectors[token_ids]
    V = V.reshape((V.shape[0] // 2, 2,) + V.shape[1:])
    V = V[:, 1] - V[:, 0]
    V = V / np.linalg.norm(V, axis=1, keepdims=True)
    print(V @ V.T)
# boy girl mommy daddy grandpa grandma

boy girl mommy daddy grandpa grandma
not found:
[[ 1.    0.09  0.02]
 [ 0.09  1.   -0.01]
 [ 0.02 -0.01  1.  ]]


In [13]:
from scipy.cluster.hierarchy import dendrogram, linkage

selected = cnt_vectors.cpu().numpy() >= 6
selected_vectors = vectors[selected]
selected_indices = np.arange(len(vectors))[selected]
n_samples = len(selected_vectors)

Z = linkage(selected_vectors, method='ward')

print(Z.shape)

(1563, 4)


In [14]:
def llf(idx):
    if idx < n_samples:
        idx = selected_indices[idx]
        return lit_model.idx2word[idx] + f'   {cnt_vectors[idx].item()}'
    else:
        return f'{idx - n_samples} {int(Z[idx - n_samples, 3])} {Z[idx - n_samples, 2]:.3f}'

p = 2000

plt.figure(figsize=(25, 0.4 * p))
_ = dendrogram(
    Z,
    #truncate_mode='lastp',
    #p=p,
    orientation='left',
    leaf_rotation=0.,
    leaf_font_size=16.,
    leaf_label_func=llf,
)